In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import qiskit
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit_aer import Aer
from qiskit import transpile
from qiskit.visualization import plot_histogram



# Chapitre 5: Vecteurs propres, valeurs propres, mesure et probabilité

## 2.1: Vecteurs propres et valeurs propres

Il arrive que multiplier un vecteur par une matrice ne change que sa longueur. Dans ce cas, on dit que c'est un vecteur propre de la matrice et le rapport des longueurs finale et initiale s'appelle la valeur propre. Un exemple très simple est:

$$ \begin{pmatrix} \lambda & 0 \\ 0 & \lambda \end{pmatrix} \begin{pmatrix} a \\ b \end{pmatrix} = \begin{pmatrix} \lambda a \\ \lambda b \end{pmatrix}  = \lambda \begin{pmatrix}  a \\ b \end{pmatrix}.$$

Mais aussi (pour des valeurs de $a$ et $b$ particulières):

$$ \begin{pmatrix} 2 & 1 \\ 2 & 3 \end{pmatrix} \begin{pmatrix} -1 \\ 1 \end{pmatrix} = \begin{pmatrix} -1 \\ 1 \end{pmatrix},$$

et

$$ \begin{pmatrix} 2 & 1 \\ 2 & 3 \end{pmatrix} \begin{pmatrix} 1 \\ 2 \end{pmatrix} = \begin{pmatrix} 4 \\ 8 \end{pmatrix} = 4 \begin{pmatrix} 1 \\ 2 \end{pmatrix},$$

où les vecteurs $(-1,1)$ et $(1,2)$ sont des vecteurs propres de la matrice et dont les valeurs propres associées sont $1$ et $4$, respectivement. Ces notions sont très importantes car, comme nous le verrons et l'utiliserons plus tard, les résultats de mesures expérimentales sont des valeurs propres de matrices que l'on appelle des observables et qui correspondent à des grandeurs physiques mesureables. Par exemple, la valeur mesurée d'une position est une valeur propre de l'opérateur position (qui est une matrice). En d'autres termes, une grandeur observable est associée à un opérateur (une matrice) dont les valeurs propres sont des résulats possibles d'une mesure expérimentale. Dit encore autrement, lorsque l'on veut mesurer une certaine grandeur (la position ou l'impulsion par exemple), le résultat de cette mesure sera l'une des valeurs propres de l'opérateur correspondant, avec une certaine probabilité.

De plus, la probabilité de mesurer une valeur propre est directement reliée au produit
scalaire entre l’état du système et le vecteur propre associé à la valeur propre
mesurée.

## 3.2: Mesure de l'état d'un qubit

Pour connaître l'état dans lequel se trouve un système, il faut effectuer une mesure. Toutefois, il n'est pas possible d'observer un état superposé. Une superposition peut uniquement être déduite du caractère probabiliste du résulat d'une mesure. 

Comme exemple, reprenons le résultat de l'application de la porte de Hadamard sur notre ket $\ket 0$ ou $\ket 1$ (noté $\ket \psi$). Pour que nous puissions lire l'information apportée par la mesure, il faut qu'elle soit stockée dans un registre que nous sommes capables de déchiffrer, et ce registre s'écrit en bits classiques. Il faut donc stocker l'information dans des bits classiques. Et un bit ne peut être que dans un seul état à la fois, soit $\ket 0$, soit $\ket 1$, et pas dans une superposition. L'encodage d'une seule mesure de l'état d'un seul qubit est stockable dans un seul bit et ce bit prendra la valeur 0 ou 1 avec une probabilité 1/2 (qui est la valeur au carré des coefficients $\pm 1/\sqrt 2$) suite au passage du qubit à travers la porte de Hadamard. Mais revenons d'abord au produit scalaire pour comprendre cet exemple avant de l'implémenter avec QISKIT.

"Mesurer un qubit" révèle l'état du qubit, ce qui est équivalent à *se demander à quel point l'état du système ressemble à l'état* $\ket 0$ ou $\ket 1$. Ceci traduit deux choses:

1) le résultat de la mesure dépend de l'état auquel le système est comparé;
2) le "à quel point" reflète la nature probabiliste du résultat de la comparaison.

Dans notre exemple, cette probabilité est donnée par un produit scalaire entre l'état final -- après le passage par la porte de Hadamard -- et le ket $\ket 0$ ou $\ket 1$:

$$P(\ket 0) = |\braket{0| H|\psi}|^2, \qquad P(\ket 1) = |\braket{1| H|\psi}|^2.$$

La notation $\braket{}$ est la façon d'écrire le produit scalaire en mécanique quantique, et il se calcule simplement par le produit scalaire des vecteurs associés aux états que l'on compare. En se souvenant que $\ket 0 = \begin{pmatrix} 1\\0\end{pmatrix}$, $\ket 1 = \begin{pmatrix} 0\\1\end{pmatrix}$ et $H\ket \psi = \frac{\ket 0 \pm \ket 1}{\sqrt 2} = \frac{1}{\sqrt 2}\begin{pmatrix}
1\\ \pm 1
\end{pmatrix}$, les différents produits scalaires sont

$$\braket{0|H|\psi} = \begin{pmatrix} 1\\0 \end{pmatrix} \cdot \frac{1}{\sqrt 2}\begin{pmatrix} 1\\\pm 1 \end{pmatrix} =\frac{1}{\sqrt 2},$$
$$\braket{1|H|\psi} = \begin{pmatrix} 0\\1 \end{pmatrix} \cdot \frac{1}{\sqrt 2}\begin{pmatrix} 1\\\pm 1 \end{pmatrix} =\pm \frac{1}{\sqrt 2}.$$

Une fois mis au carré, on obtient la probabilité de mesurer le qubit dans l'état $\ket 0$ ou $\ket 1$, qui, ici, vaut 1/2. Mettons à présent cela en pratique avec QISKIT.

In [ ]:
# 1: Create a 1-qubit circuit and 1 classical bit to sore the measurement output
circuit = QuantumCircuit(1,1)

# 2: Definir l'état initial et l'ajouter au début du circuit:
etat_initial = [1, 0]  # l'état |0>
circuit.initialize(etat_initial, 0)

# 3: Ajouter une porte de Hadamard au circuit
circuit.h(0)

# 4: Appliquer le circuit sur l'éat initial
etat_final = Statevector.from_instruction(circuit)
print("Final statevector:", etat_final)
display(etat_final.draw("latex"))

# 5: Mesurer l'état du système (|0> ou |1>)
circuit.measure(0,0)  # mesure du qubit et stockage dans le bit classique

circuit.draw('mpl')

On peut mesurer l'état dans lequel se trouve le qubit. Notons que nous n'avons pas vu comment faire tourner du code sur un ordinateur quantique. Avant ça, nous vous présentons une méthode adaptée aux ordinateurs classiques qui imite le fonctionnement un ordinateur quantique. Ceci revient à générer artifiellement de l'aléatoire dans le processus de mesure (intrinsèquement présent dans un ordinateur quantique). Il est courant de faire cela parce que l'accès aux ordinateurs quantiques est limité dans le temps et peut s'avérer coûteux. Afin d'explorer l'informatique quantique et plus largement de tester des algorithmes, il est donc très pratique d'utiliser des simulateurs.

Ci-dessous, appliquons d'abord le processus de mesure une seule fois. Le résultat final va donner soit 0, soit 1. En faisant tourner le code de cette même cellule, équivalent à faire passer le qubit plusieurs fois dans notre petit circuit, le résulat va parfois être 0, parfois 1.

In [ ]:
# 1: Simuler la mesure
simulateur = Aer.get_backend('aer_simulator')

# Parcours du circuit une fois:
n = 1 # nombre de parcours du circuit avec le même qubit initial
resultats = simulateur.run(transpile(circuit, simulateur), shots=n).result()
comptage = resultats.get_counts()

# Step 3: show results
print(comptage)
plot_histogram(comptage)

On peut choisir un nombre arbitraire de parcours du circuit et compter le nombre de fois où le 0 et le 1 sont mesurés. On affiche les résultats de mesure dans un histogramme. On observe que sur 1000 mesures, le 0 et le 1 ont été mesurés environ 500 fois, ce qui rappelle leur probabilité proche de 1/2. 

In [ ]:
# 1: Simuler la mesure
simulateur = Aer.get_backend('aer_simulator')

# 2: Parcours du circuit "n" fois:
n = 1000 # nombre de parcours du circuit avec le même qubit initial
resultats = simulateur.run(transpile(circuit, simulateur), shots=n).result()
comptage = resultats.get_counts()

# 3: Histogramme du comptage
print(comptage)
plot_histogram(comptage)

Maintenant que nous avons appris à construire des circuits à 1 qubit et à mesurer leur état, passons à des circuits à 2 qubits.